In [24]:
import pandas as pd, numpy as np
from rolling_framework import Machine

import warnings
warnings.filterwarnings('ignore')

### Data Loading

In [25]:
period = '19712018'

#%% Target variables : excess returns
y = pd.read_csv(f'data/{period}/exrets_{period}.csv', index_col='Time')             # Target : Excess Return (Bond Risk Premia)
y = y[['xr_2','xr_5','xr_7','xr_10']]

#%% Predictors (Features)
FWDS = pd.read_csv(f'data/{period}/fwds_{period}.csv', index_col='Time')            # Forward Rates
MACV = pd.read_csv(f'data/{period}/MacroFactors_{period}.csv', index_col='Time')    # Ludvigson and Ng (2009), Macrovariables
LSC = pd.read_csv(f'data/{period}/lsc_{period}.csv', index_col='Time')              # Level, Slope and Curvature
# YLV = pd.read_csv(f'data/{period}/yl_{period}.csv', index_col='Time')               # yield-level
RVOL = pd.read_csv(f'data/{period}/real_vol_{period}.csv', index_col='Time')        # Realised Volatility 10yr
#IVOL = pd.read_csv(f'data/{period}/imp_vol_{period}.csv', index_col='Time')        # Implied Volatility (Work for 1990~)
CP = pd.read_csv(f'data/{period}/cp_{period}.csv', index_col='Time')                # Cochrane-Piazessi Factor
# YALL = pd.read_csv(f'data/{period}/yl_all_{period}.csv', index_col='Time')          # 12 m, 24 m, ... 120 m
# YMAX = pd.read_csv(f'data/{period}/yl_max_{period}.csv', index_col='Time')          # 1m ... 120 m

F6 = MACV[['F1','F2','F3','F4','F8','F1^3']]
SL = LSC[['slope']]
LV = LSC[['level']]
CU = LSC[['curvature']]


AGG = pd.concat([CP, FWDS], axis=1)

In [40]:
# ────────────────────────────
# Random-Forest
# ────────────────────────────
param_grid_rf = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5,1],
}

# ────────────────────────────
# Extra-Trees
# ────────────────────────────
param_grid_et = {
    "model__estimator__n_estimators":      [300],
    "model__estimator__max_depth":         [2, 8],
    "model__estimator__min_samples_split": [2, 4],
    "model__estimator__min_samples_leaf":  [1, 2, 4],
    "model__estimator__max_features":      [0.25, 0.5, 1],
}

# ────────────────────────────
# XGBoost  (XGBRegressor 사용 가정)
# ────────────────────────────
param_grid_xgb = {
    "model__estimator__n_estimators":  [300],
    "model__estimator__max_depth":     [2, 4],
    "model__estimator__learning_rate": [0.01],
    "model__estimator__subsample":     [0.7, 0.5],
    "model__estimator__reg_lambda":    [0.1, 1.0],
}

param_grid_lasso = {'reg__alpha': [0.001, 1.0 ]}    
param_grid_ridge      = {'reg__alpha': [0.001, 10]}
param_grid_elasticnet = {
    'reg__alpha':   [0.01, 0.1, 1, 10],
    'reg__l1_ratio':[0.1, 0.3, 0.5],
}

In [4]:
w_custom = pd.Series(
    {
        "xr_2" : 2,   # 2Y 듀레이션
        "xr_5" : 5,   # 5Y 듀레이션
        "xr_7" : 7,   # 7Y 듀레이션
        "xr_10": 10,   # 10Y 듀레이션
    },
    name="w"
)

cols_dw = w_custom.index.tolist()
# 합이 1이 되도록 정규화
w_custom = w_custom / w_custom.sum()


### OLS, LASSO, ELASTICNET

In [5]:
ols_1 = Machine(SL, y, 'OLS')
ols_1.training()
ols_2 = Machine(CP,y, 'OLS')
ols_2.training()
ols_3 = Machine(F6, y, 'OLS')
ols_3.training()

print(ols_1.R2OOS())
print('ew :', ols_1.r2_oos_portfolio())
print('dw :', ols_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_2.R2OOS())
print('ew :', ols_2.r2_oos_portfolio())
print('dw :', ols_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ols_3.R2OOS())
print('ew :', ols_3.r2_oos_portfolio())
print('dw :', ols_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

OLS rolling: 100%|██████████| 360/360 [00:05<00:00, 61.16it/s]


xr_2    -0.123412
xr_5     0.001880
xr_7     0.058773
xr_10    0.143233
dtype: float64
ew : -0.06423556982957912
dw : -0.04582135336419335
xr_2    -0.087916
xr_5     0.024065
xr_7     0.053338
xr_10    0.090852
dtype: float64
ew : -0.07648799417278496
dw : -0.07404469980017625
xr_2    -0.234431
xr_5     0.030841
xr_7     0.054242
xr_10    0.113406
dtype: float64
ew : -0.06845732307782759
dw : -0.05709454329788777


In [41]:
ridge_1 = Machine(SL, y, 'Penalized', option='ridge', params_grid=param_grid_ridge)
ridge_1.training()
ridge_2 = Machine(CP, y, 'Penalized', option='ridge', params_grid=param_grid_ridge)
ridge_2.training()
ridge_3 = Machine(F6, y, 'Penalized', option='ridge', params_grid=param_grid_ridge)
ridge_3.training()


print(ridge_1.R2OOS())
print('ew :', ridge_1.r2_oos_portfolio())
print('dw :', ridge_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_2.R2OOS())
print('ew :', ridge_2.r2_oos_portfolio())
print('dw :', ridge_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(ridge_3.R2OOS())
print('ew :', ridge_3.r2_oos_portfolio())
print('dw :', ridge_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 360/360 [00:06<00:00, 59.85it/s]


xr_2    -0.123884
xr_5     0.001005
xr_7     0.058244
xr_10    0.143068
dtype: float64
ew : -0.06487456680729875
dw : -0.046331078179065344
xr_2    -0.082743
xr_5     0.028335
xr_7     0.058308
xr_10    0.095801
dtype: float64
ew : -0.07069507783680873
dw : -0.0682626818669958
xr_2    -0.220154
xr_5     0.034766
xr_7     0.060617
xr_10    0.118650
dtype: float64
ew : -0.06159499411048608
dw : -0.05060802281011023


In [9]:
lasso_1 = Machine(SL, y, 'Penalized', option='lasso', params_grid=param_grid_lasso)
lasso_1.training()
lasso_2 = Machine(CP, y, 'Penalized', option='lasso', params_grid=param_grid_lasso)
lasso_2.training()
lasso_3 = Machine(F6, y, 'Penalized', option='lasso', params_grid=param_grid_lasso)
lasso_3.training()


print(lasso_1.R2OOS())
print('ew :', lasso_1.r2_oos_portfolio())
print('dw :', lasso_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_2.R2OOS())
print('ew :', lasso_2.r2_oos_portfolio())
print('dw :', lasso_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(lasso_3.R2OOS())
print('ew :', lasso_3.r2_oos_portfolio())
print('dw :', lasso_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 360/360 [00:06<00:00, 57.37it/s]


xr_2    -0.181314
xr_5    -0.074675
xr_7     0.023785
xr_10    0.129010
dtype: float64
ew : -0.10432153991969217
dw : -0.07787138039489383
xr_2    -0.020068
xr_5     0.039852
xr_7     0.068974
xr_10    0.104184
dtype: float64
ew : -0.05326590134866649
dw : -0.05494517411991784
xr_2    -0.398807
xr_5    -0.077570
xr_7    -0.006363
xr_10    0.061337
dtype: float64
ew : -0.15487192898845925
dw : -0.1332750391015518


In [14]:
elasticnet_1 = Machine(SL, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_1.training()

elasticnet_2 = Machine(CP, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_2.training()

elasticnet_3 = Machine(F6, y, 'Penalized', option='elasticnet', params_grid=param_grid_elasticnet)
elasticnet_3.training()


print(elasticnet_1.R2OOS())
print('ew :', elasticnet_1.r2_oos_portfolio())
print('dw :', elasticnet_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_2.R2OOS())
print('ew :', elasticnet_2.r2_oos_portfolio())
print('dw :', elasticnet_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(elasticnet_3.R2OOS())
print('ew :', elasticnet_3.r2_oos_portfolio())
print('dw :', elasticnet_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Penalized rolling: 100%|██████████| 360/360 [00:11<00:00, 30.52it/s]


xr_2    -0.175665
xr_5    -0.089630
xr_7    -0.028150
xr_10    0.063809
dtype: float64
ew : -0.1658429596938562
dw : -0.14602176236112752
xr_2    -0.049290
xr_5     0.019847
xr_7     0.053159
xr_10    0.098711
dtype: float64
ew : -0.07074550033640814
dw : -0.06899130940175424
xr_2    -0.234712
xr_5    -0.089215
xr_7    -0.054463
xr_10   -0.005559
dtype: float64
ew : -0.20162131707054187
dw : -0.19458831277632793


### Tree

In [21]:
rf_1 = Machine(SL, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf)
rf_1.training()
rf_2 = Machine(CP, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf)
rf_2.training()
rf_3 = Machine(F6, y, 'Tree', option = 'RandomForest', params_grid=param_grid_rf)
rf_3.training()

print(rf_1.R2OOS())
print('ew :', rf_1.r2_oos_portfolio())
print('dw :', rf_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_2.R2OOS())
print('ew :', rf_2.r2_oos_portfolio())
print('dw :', rf_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(rf_3.R2OOS())
print('ew :', rf_3.r2_oos_portfolio())
print('dw :', rf_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 360/360 [1:16:14<00:00, 12.71s/it]


xr_2    -0.322312
xr_5    -0.194365
xr_7    -0.143179
xr_10   -0.086767
dtype: float64
ew : -0.30853092732177645
dw : -0.29695440848991783
xr_2    -0.291911
xr_5    -0.153129
xr_7    -0.158045
xr_10   -0.134921
dtype: float64
ew : -0.3218969704203005
dw : -0.32570452297613106
xr_2    -0.105528
xr_5    -0.026950
xr_7    -0.018339
xr_10    0.016035
dtype: float64
ew : -0.14754431505582288
dw : -0.15242240827704445


In [22]:
et_1 = Machine(SL, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et)
et_1.training()
et_2 = Machine(CP, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et)
et_2.training()
et_3 = Machine(F6, y, 'Tree', option='ExtremeTrees', params_grid=param_grid_et)
et_3.training()


print(et_1.R2OOS())
print('ew :', et_1.r2_oos_portfolio()) 
print('dw :', et_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_2.R2OOS())
print('ew :', et_2.r2_oos_portfolio())
print('dw :', et_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(et_3.R2OOS())
print('ew :', et_3.r2_oos_portfolio())
print('dw :', et_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


Tree rolling: 100%|██████████| 360/360 [26:54<00:00,  4.49s/it]


xr_2    -0.255686
xr_5    -0.130337
xr_7    -0.057983
xr_10    0.036738
dtype: float64
ew : -0.2003841169562488
dw : -0.17843241262302434
xr_2    -0.064640
xr_5     0.006700
xr_7     0.031542
xr_10    0.059722
dtype: float64
ew : -0.1013521608194965
dw : -0.10393268123724053
xr_2    -0.103369
xr_5    -0.008458
xr_7     0.011706
xr_10    0.034229
dtype: float64
ew : -0.12481393878216607
dw : -0.12847741190647888


In [30]:
xgb_1 = Machine(SL, y, 'Tree', option='XGB', params_grid=param_grid_xgb)
xgb_1.training()
xgb_2 = Machine(CP, y, 'Tree', option='XGB', params_grid=param_grid_xgb)
xgb_2.training()
xgb_3 = Machine(F6, y, 'Tree', option='XGB', params_grid=param_grid_xgb)
xgb_3.training()

print(xgb_1.R2OOS())
print('ew :', xgb_1.r2_oos_portfolio())
print('dw :', xgb_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_2.R2OOS())
print('ew :', xgb_2.r2_oos_portfolio())
print('dw :', xgb_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

print(xgb_3.R2OOS())
print('ew :', xgb_3.r2_oos_portfolio())
print('dw :', xgb_3.r2_oos_portfolio(cols = cols_dw, weights=w_custom))

Tree rolling: 100%|██████████| 360/360 [02:53<00:00,  2.07it/s]


xr_2    -0.285058
xr_5    -0.212481
xr_7    -0.151518
xr_10   -0.071317
dtype: float64
ew : -0.30817883308055993
dw : -0.29332815913482446
xr_2    -0.208705
xr_5    -0.065617
xr_7    -0.055354
xr_10   -0.031152
dtype: float64
ew : -0.2056985326144063
dw : -0.20730080010911078
xr_2    -0.056118
xr_5     0.013407
xr_7     0.011996
xr_10    0.037086
dtype: float64
ew : -0.11310894716789544
dw : -0.12042677304508231


### DNN : FWDs ONly

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_1 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    forecast_horizon=12,
    random_state=15
)
dnn_1.training()
print(dnn_1.R2OOS())
print('ew :', dnn_1.r2_oos_portfolio())
print('dw :', dnn_1.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


### DNN : FWDS + MACRO

In [ ]:
param_grid = {
    "dnn__module__hidden": [(3,3)],    
    "dnn__module__dropout": [0.2],
    "dnn__lr": [0.01],
    "dnn__optimizer__weight_decay": [0.0005],
}

dnn_2 = Machine(
    FWDS, y,
    model_type="DNN",                  
    params_grid=param_grid,
    forecast_horizon=12,
    random_state=15
)
dnn_2.training()
print(dnn_2.R2OOS())
print('ew :', dnn_2.r2_oos_portfolio())
print('dw :', dnn_2.r2_oos_portfolio(cols = cols_dw, weights=w_custom))


In [ ]:
AGG = pd.concat([MACV, CP], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(1)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG.columns if c.startswith("F")]
grp2_cols = [c for c in AGG.columns if c not in grp1_cols]

dnn_d1 = Machine(
    AGG, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    params_grid=param_grid,
    forecast_horizon=12
)
dnn_d1.training()
print(dnn_d1.R2OOS())

In [ ]:
AGG2 = pd.concat([MACV, FWDS], axis=1)

# ── 예시: dual-branch MLP 하이퍼파라미터 그리드 ─────────────────────────
param_grid = {
    # ── Branch-1 Macro
    "dnn__module__hidden1": [(32,)],    # 얕은 vs. 두 층
    "dnn__module__drop1":   [0.2],       # 드롭아웃 비율

    # ── Branch-2 FWDS : CP
    "dnn__module__hidden2": [(3,3)],
    "dnn__module__drop2":   [0.2],

    # ── 공통 optimizer & regularization ────────────────────────────────
    "dnn__optimizer__lr":           [0.001],
    "dnn__optimizer__weight_decay": [0.0005]
}

grp1_cols = [c for c in AGG2.columns if c.startswith("F")]
grp2_cols = [c for c in AGG2.columns if c not in grp1_cols]

dnn_d2 = Machine(
    AGG2, y,
    model_type="DNN_DUAL",
    option={"grp1": grp1_cols, "grp2": grp2_cols},
    params_grid=param_grid,
    forecast_horizon=12
)
dnn_d2.training()
print(dnn_d2.R2OOS())